# JS: Async

## Check whether is promise

- https://github.com/sudheerj/javascript-interview-questions#how-do-you-check-an-object-is-a-promise-or-not

In [14]:
function isPromise(value) {
  return Boolean(value && typeof value.then === "function");
}

console.log(isPromise(123));
console.log(isPromise(new Promise(function (resolve, reject) {
  resolve();
})));
console.log(isPromise(new Date()));

false
true
false


## How do you prevent promises swallowing errors

- https://github.com/sudheerj/javascript-interview-questions#how-do-you-prevent-promises-swallowing-errors

In [4]:
Promise.resolve("promised value").then(function () {
  throw new Error("error 1");
});

Promise { <pending> }


UnhandledPromiseRejection: Error: error 1
    at evalmachine.<anonymous>:4:11


In [6]:
Promise.reject("error value").catch(function () {
  throw new Error("error 2");
});

Promise { <pending> }


UnhandledPromiseRejection: Error: error 2
    at evalmachine.<anonymous>:4:11


In [7]:
new Promise(function (resolve, reject) {
  throw new Error("error 3");
});

Promise {
  <rejected> Error: error 3
      at evalmachine.<anonymous>:4:11
      at new Promise (<anonymous>)
      at evalmachine.<anonymous>:3:22
      at evalmachine.<anonymous>:7:3
      at sigintHandlersWrap (vm.js:273:12)
      at Script.runInThisContext (vm.js:131:14)
      at Object.runInThisContext (vm.js:310:38)
      at Object.execute (/home/d9k/.nvm/versions/node/v14.16.1/lib/node_modules/tslab/dist/executor.js:160:38)
      at JupyterHandlerImpl.handleExecuteImpl (/home/d9k/.nvm/versions/node/v14.16.1/lib/node_modules/tslab/dist/jupyter.js:223:38)
      at /home/d9k/.nvm/versions/node/v14.16.1/lib/node_modules/tslab/dist/jupyter.js:181:57
}


UnhandledPromiseRejection: Error: error 3
    at evalmachine.<anonymous>:4:11
    at new Promise (<anonymous>)
    at evalmachine.<anonymous>:3:22
    at evalmachine.<anonymous>:7:3
    at sigintHandlersWrap (vm.js:273:12)
    at Script.runInThisContext (vm.js:131:14)
    at Object.runInThisContext (vm.js:310:38)
    at Object.execute (/home/d9k/.nvm/versions/node/v14.16.1/lib/node_modules/tslab/dist/executor.js:160:38)
    at JupyterHandlerImpl.handleExecuteImpl (/home/d9k/.nvm/versions/node/v14.16.1/lib/node_modules/tslab/dist/jupyter.js:223:38)
    at /home/d9k/.nvm/versions/node/v14.16.1/lib/node_modules/tslab/dist/jupyter.js:181:57


TODO all works as is

## Async function

Handling reject:

In [24]:
const promiseToBeRejected = new Promise((resolve, reject) => { reject('reject value') });

const myAsyncFnUnhandled = async () => {
  await promiseToBeRejected;
}

myAsyncFnUnhandled();

Promise { <pending> }


UnhandledPromiseRejection: reject value


In [26]:
const myAsyncFnCatch = async () => {
    try {
        await promiseToBeRejected;
    } catch (e) {
        console.log('There was successfully handled error:', e)
    }
}

myAsyncFnCatch();

Promise { <pending> }
There was successfully handled error: reject value


## Console output of promise

In [29]:
async function func() {
  return 10;
}
console.log(func());

Promise { 10 }


## Promise, timeout order

https://github.com/sudheerj/javascript-interview-questions#65-what-is-the-output-order-of-below-code

In [34]:
setTimeout(() => {
  console.log("1");
}, 0);

Promise.resolve("hello").then(() => console.log("2"));

console.log("3");

setImmediate(() => {
  console.log(4);
});

process.nextTick(() => {
  console.log(5);
});

3
2
5
4
1


## Promise.all(), Promise.allSettled()

https://github.com/sudheerj/javascript-interview-questions#77-what-is-the-output-of-below-code

In [5]:
var promiseResolve1 = new Promise((resolve, reject) => setTimeout(() => resolve(1), 2000));
var promiseResolve2 = new Promise((resolve, reject) => setTimeout(() => resolve(2), 1000));
var promiseReject = new Promise((resolve, reject) => setTimeout(() => reject(1), 2000));

Promise.all([promiseResolve1, promiseReject]).then((data) => console.log(data));

Promise { <pending> }


UnhandledPromiseRejection: 1


In [8]:
Promise.all([promiseResolve1, promiseResolve2]).then((data) => console.log(data));

Promise { <pending> }
[ 1, 2 ]


In [9]:
Promise.allSettled([promiseResolve1, promiseReject]).then((data) => console.log(data));

Promise { <pending> }
[
  { status: 'fulfilled', value: 1 },
  { status: 'rejected', reason: 1 }
]


Promise is native in ES6 and it will be moved to Job queue which has high priority than callback queue in the execution order. 

## setTimeout() + try/catch

https://github.com/sudheerj/javascript-interview-questions#78-what-is-the-output-of-below-code

In [11]:
try {
  setTimeout(() => {
    console.log("try block");
    throw new Error(`An exception is thrown`);
  }, 1000);
} catch (err) {
  console.log("Error: ", err);
}

try block


UncaughtException: Error: An exception is thrown
    at Timeout._onTimeout (evalmachine.<anonymous>:5:15)
    at listOnTimeout (internal/timers.js:554:17)
    at processTimers (internal/timers.js:497:7)


try/catch can't intercept timeout errors

Fixing:

In [16]:
void setTimeout(() => {
  try {
    console.log("try block");
    throw new Error(`An exception is thrown`);
  } catch (err) {
    console.log("Successfully handled: ", err);
  }
}, 1000);

try block
Successfully handled:  Error: An exception is thrown
    at Timeout._onTimeout (evalmachine.<anonymous>:6:15)
    at listOnTimeout (internal/timers.js:554:17)
    at processTimers (internal/timers.js:497:7)


## Promise constructor

is synchronous

In [10]:
void setTimeout(()=>{
    console.log("timeOut");
}, 0)

console.log(1);

void new Promise(resolve=>{
    console.log("Promise constructor");
    void setTimeout(()=>{
        console.log("777");
        resolve()
    },0)
}).then(()=>{
    console.log("then1");
}).then(()=>{
    console.log("then2");
})

console.log(4);

void setTimeout(()=>{
    console.log("timeOut2");
},0)

1
Promise constructor
4
timeOut
777
then1
then2
timeOut2
